In [33]:
import requests
import pandas as pd
from pprint import pprint
import pycountry

In [2]:
OPENAQ_API_KEY = "2df1e1ad492dbcc9293e82253f0ddffca796f334d424d0c1765e827c87a219ea"

BASE_URL = "https://api.openaq.org/v3/"

In [3]:
headers = {
    "X-API-Key": OPENAQ_API_KEY
}

In [11]:
endpoint_parameters = f"{BASE_URL}parameters"
endpoint_locations = f"{BASE_URL}locations"
endpoint_countries = f"{BASE_URL}countries"

In [12]:
major_us_cities = [
    "Los Angeles", "New York", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"
]


In [38]:
list(pycountry.countries)

[Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533'),
 Country(alpha_2='AF', alpha_3='AFG', flag='🇦🇫', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan'),
 Country(alpha_2='AO', alpha_3='AGO', flag='🇦🇴', name='Angola', numeric='024', official_name='Republic of Angola'),
 Country(alpha_2='AI', alpha_3='AIA', flag='🇦🇮', name='Anguilla', numeric='660'),
 Country(alpha_2='AX', alpha_3='ALA', flag='🇦🇽', name='Åland Islands', numeric='248'),
 Country(alpha_2='AL', alpha_3='ALB', flag='🇦🇱', name='Albania', numeric='008', official_name='Republic of Albania'),
 Country(alpha_2='AD', alpha_3='AND', flag='🇦🇩', name='Andorra', numeric='020', official_name='Principality of Andorra'),
 Country(alpha_2='AE', alpha_3='ARE', flag='🇦🇪', name='United Arab Emirates', numeric='784'),
 Country(alpha_2='AR', alpha_3='ARG', flag='🇦🇷', name='Argentina', numeric='032', official_name='Argentine Republic'),
 Country(alpha_2='AM', alpha_3='ARM', flag='🇦🇲', 

In [ ]:
country_dict = {country.name: country.numeric for country in list(pycountry.countries)}

pprint(country_dict)

{'Afghanistan': '004',
 'Albania': '008',
 'Algeria': '012',
 'American Samoa': '016',
 'Andorra': '020',
 'Angola': '024',
 'Anguilla': '660',
 'Antarctica': '010',
 'Antigua and Barbuda': '028',
 'Argentina': '032',
 'Armenia': '051',
 'Aruba': '533',
 'Australia': '036',
 'Austria': '040',
 'Azerbaijan': '031',
 'Bahamas': '044',
 'Bahrain': '048',
 'Bangladesh': '050',
 'Barbados': '052',
 'Belarus': '112',
 'Belgium': '056',
 'Belize': '084',
 'Benin': '204',
 'Bermuda': '060',
 'Bhutan': '064',
 'Bolivia, Plurinational State of': '068',
 'Bonaire, Sint Eustatius and Saba': '535',
 'Bosnia and Herzegovina': '070',
 'Botswana': '072',
 'Bouvet Island': '074',
 'Brazil': '076',
 'British Indian Ocean Territory': '086',
 'Brunei Darussalam': '096',
 'Bulgaria': '100',
 'Burkina Faso': '854',
 'Burundi': '108',
 'Cabo Verde': '132',
 'Cambodia': '116',
 'Cameroon': '120',
 'Canada': '124',
 'Cayman Islands': '136',
 'Central African Republic': '140',
 'Chad': '148',
 'Chile': '152',
 

In [13]:
parameters_params = {
    "limit": 10,
}

In [32]:
params = {"countries_id": [79], "limit": 100}

response = requests.get(endpoint_locations, params=params, headers=headers)

locations = response.json()
pprint(locations)

{'meta': {'found': '>100',
          'limit': 100,
          'name': 'openaq-api',
          'page': 1,
          'website': '/'},
 'results': [{'bounds': [-0.06821799979396657,
                         51.59929999967941,
                         -0.06821799979396657,
                         51.59929999967941],
              'coordinates': {'latitude': 51.59929999967941,
                              'longitude': -0.06821799979396657},
              'country': {'code': 'GB', 'id': 79, 'name': 'United Kingdom'},
              'datetimeFirst': {'local': '2016-01-30T01:00:00Z',
                                'utc': '2016-01-30T01:00:00Z'},
              'datetimeLast': {'local': '2025-05-21T09:00:00+01:00',
                               'utc': '2025-05-21T08:00:00Z'},
              'distance': None,
              'id': 60,
              'instruments': [{'id': 2, 'name': 'Government Monitor'}],
              'isMobile': False,
              'isMonitor': True,
              'licenses': [

In [27]:
from requests import Response


location_id = 2416

params = {"limit": 10}

response = requests.get(f"{endpoint_locations}/{location_id}/sensors", params=params, headers=headers)

pprint(response.json())

{'meta': {'found': 1,
          'limit': 100,
          'name': 'openaq-api',
          'page': 1,
          'website': '/'},
 'results': [{'coverage': {'datetimeFrom': {'local': '2016-02-27T21:00:00Z',
                                            'utc': '2016-02-27T21:00:00Z'},
                           'datetimeTo': {'local': '2025-05-21T09:00:00+01:00',
                                          'utc': '2025-05-21T08:00:00Z'},
                           'expectedCount': 1,
                           'expectedInterval': '01:00:00',
                           'observedCount': 52033,
                           'observedInterval': '52033:00:00',
                           'percentComplete': 5203300.0,
                           'percentCoverage': 5203300.0},
              'datetimeFirst': {'local': '2016-02-27T21:00:00Z',
                                'utc': '2016-02-27T21:00:00Z'},
              'datetimeLast': {'local': '2025-05-21T09:00:00+01:00',
                               'utc

get_parameters is used for getting the unit used for measuring respective data

In [17]:
def get_parameters():
    response = requests.get(endpoint_parameters, headers=headers, params=parameters_params)
    if response.status_code == 200:
        data = response.json()
        para_df = pd.DataFrame(data['results'])
        print("Parameters:")
        print(para_df)
        return para_df
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return pd.DataFrame()

In [18]:
def get_locations():
    all_cities_with_data = []

    for city in major_us_cities:
        params = {
            "country": "US", "limit": 5, "page": 1
            }
        response = requests.get(endpoint_locations, headers=headers, params=params)
        data = response.json()
    
        if "results" in data:
            city_data = data['results']
            if city_data:
                city_df = pd.DataFrame(city_data)
                city_df['city'] = city
                all_cities_with_data.append(city_df)
            else:
                print(f"No data found for {city}.")

        else:
            # print(f"Error: {response.status_code} - {response.text}")
            pass
    if all_cities_with_data:
        all_cities_df = pd.concat(all_cities_with_data, ignore_index=True)
        print("Locations:")
        print(all_cities_df)
    else:
        print("No locations found for the specified cities.")
        return pd.DataFrame()

In [19]:
def main():
    get_parameters()
    get_locations()

In [20]:
if __name__ == "__main__":
    main()

Parameters:
   id  name  units displayName  \
0   1  pm10  µg/m³        PM10   
1   2  pm25  µg/m³       PM2.5   
2   3    o3  µg/m³     O₃ mass   
3   4    co  µg/m³     CO mass   
4   5   no2  µg/m³    NO₂ mass   
5   6   so2  µg/m³    SO₂ mass   
6   7   no2    ppm         NO₂   
7   8    co    ppm          CO   
8   9   so2    ppm         SO₂   
9  10    o3    ppm          O₃   

                                         description  
0  Particulate matter less than 10 micrometers in...  
1  Particulate matter less than 2.5 micrometers i...  
2                           Ozone mass concentration  
3                 Carbon Monoxide mass concentration  
4                Nitrogen Dioxide mass concentration  
5                  Sulfur Dioxide mass concentration  
6                     Nitrogen Dioxide concentration  
7                      Carbon Monoxide concentration  
8                       Sulfur Dioxide concentration  
9                                Ozone concentration  
Location

Code not working. Likely due to the fact that JSON for locations uses locality, and not city.